In [2]:

import os
import os.path as pth
import cv2
import matplotlib.image as img
import numpy as np
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline 
from PIL import Image 

print('ready')

ready


In [3]:
df_cloth_attr = pd.read_csv('./data/for_prediction/labeling/list_attr_cloth.txt', sep='\s+\s+', skiprows=1)
display(df_cloth_attr.head())
df_cloth_attr_path = pd.read_csv('./data/for_prediction/labeling/list_attr_img.txt', sep='\s+', skiprows=2, header = None)
display(df_cloth_attr_path.head())
df_cloth_category = pd.read_csv('./data/for_prediction/labeling/list_category_cloth.txt', sep='\s+\s+', skiprows=1)
display(df_cloth_category.head())
df_cloth_category_path = pd.read_csv('./data/for_prediction/labeling/list_category_img.txt', sep='\s+', skiprows=1)
display(df_cloth_category_path.head())
df_cloth_bbox = pd.read_csv('./data/for_prediction/labeling/list_bbox.txt', sep='\s+', skiprows=1)
display(df_cloth_bbox.head())

/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,attribute_name,attribute_type
0,a-line,3
1,abstract,1
2,abstract chevron,1
3,abstract chevron print,1
4,abstract diamond,1


,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,1000
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


,category_name,category_type
0,Anorak,1
1,Blazer,1
2,Blouse,1
3,Bomber,1
4,Button-Down,1


,image_name,category_label
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,3
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,3
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,3
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,3
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,3


,image_name,x_1,y_1,x_2,y_2
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,72,79,232,273
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,67,59,155,161
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,65,65,156,200
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,51,62,167,182
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,46,88,166,262


In [4]:
# 타입 별 속성들 확인 1,2,3,4,5
type_number = 5

# print(df_cloth_attr.loc[df_cloth_attr['attribute_type'] == type_number])
print(df_cloth_attr.loc[df_cloth_attr['attribute_type'] == type_number]['attribute_name'])

attr_list = []

for i in df_cloth_attr.loc[df_cloth_attr['attribute_type'] == type_number]['attribute_name']:
    attr_list.append(i)
    
print(attr_list)

16     americana
17       angeles
23           art
27      athletic
28        audrey
         ...    
986         yoga
987         yoke
988         york
989        youth
990     zeppelin
Name: attribute_name, Length: 230, dtype: object
['americana', 'angeles', 'art', 'athletic', 'audrey', 'babe', 'babydoll', 'barbie', 'baseball', 'basic', 'basquiat', 'beach', 'beatles', 'bed', 'bella', 'bike', 'biker', 'blah', 'blurred', 'boho', 'bold', 'boyfriend', 'brooklyn', 'brooklyn nets', 'california', 'camera', 'candy', 'cardio', 'cat', 'chic', 'cities', 'city', 'civil', 'classic', 'coast', 'coffee', 'cute', 'dainty', 'daring', 'dark', 'darling', 'defyant', 'desert', 'destroyed', 'devil', 'doll', 'doodle', 'dream', 'dreamcatcher', 'dreamer', 'dynamite', 'eagle', 'edge', 'eiffel', 'elegant', 'enchanted', 'ethereal', 'everyday', 'fan', 'fancy', 'festive', 'field', 'fisherman', 'flawless', 'flirty', 'floyd', 'fox', 'france', 'free spirit', 'fresh', 'frida', 'galaxy', 'garden', 'garden party', 'genu

In [15]:

basic_path = './data/for_prediction/'

def make_crop_easy(file_path, my_bbox_df):
    img = Image.open(file_path)
    crop_square = (my_bbox_df['x_1'], my_bbox_df['y_1'], my_bbox_df['x_2'], my_bbox_df['y_2'])
    
    crop_img = img.crop(crop_square)
    np_crop_img = np.array(crop_img)
    
    return np_crop_img


my_style_dict = {}

for i in attr_list:
    list_of_attr = []
    for j in df_cloth_attr_path.index[df_cloth_attr_path[df_cloth_attr.index[df_cloth_attr['attribute_name'] == i][0] + 1] == 1]:
        list_of_attr.append(j)
        # make_crop_easy(basic_path + df_cloth_attr_path.iloc[j][0], df_cloth_bbox.iloc[j])
    print(i , ' : ', len(list_of_attr))
    my_style_dict[i] = list_of_attr
    
print(my_style_dict)

americana  :  103
angeles  :  533
art  :  491
athletic  :  1017
audrey  :  224
babe  :  196
babydoll  :  1330
barbie  :  276
baseball  :  1880
basic  :  1477
basquiat  :  128
beach  :  1724
beatles  :  222
bed  :  86
bella  :  387
bike  :  156
biker  :  621
blah  :  56
blurred  :  95
boho  :  2127
bold  :  514
boyfriend  :  2001
brooklyn  :  579
brooklyn nets  :  358
california  :  614
camera  :  103
candy  :  356
cardio  :  55
cat  :  603
chic  :  2099
cities  :  83
city  :  939
civil  :  176
classic  :  7184
coast  :  306
coffee  :  287
cute  :  2550
dainty  :  242
daring  :  113
dark  :  1864
darling  :  258
defyant  :  52
desert  :  207
destroyed  :  868
devil  :  75
doll  :  371
doodle  :  77
dream  :  525
dreamcatcher  :  57
dreamer  :  84
dynamite  :  60
eagle  :  766
edge  :  500
eiffel  :  83
elegant  :  1428
enchanted  :  136
ethereal  :  80
everyday  :  841
fan  :  136
fancy  :  355
festive  :  88
field  :  165
fisherman  :  218
flawless  :  91
flirty  :  232
floyd  :  91
fo

In [5]:
print(my_style_dict['sporty'])

[1948, 2980, 7372, 10471, 15326, 16078, 16092, 16588, 20183, 26386, 29847, 32822, 32858, 32986, 33436, 35010, 36765, 37001, 37196, 39057, 40141, 40566, 41512, 41548, 42679, 43558, 43616, 43698, 43953, 45106, 45107, 45962, 46138, 46154, 46172, 47892, 48167, 48611, 48802, 49345, 49617, 55625, 55981, 56041, 56070, 56196, 56343, 56376, 56382, 56393, 56402, 57795, 58038, 58059, 58077, 58441, 58759, 59185, 61718, 63266, 63941, 63956, 64015, 64033, 65570, 65897, 66010, 66282, 66413, 67440, 67444, 68021, 71891, 71910, 71957, 72394, 72451, 72490, 73722, 74064, 77992, 78605, 81397, 81860, 82108, 84419, 85268, 89614, 90063, 90502, 90530, 91613, 91640, 91796, 92214, 92830, 93746, 93758, 94165, 94897, 95170, 96659, 96690, 98962, 103654, 106006, 108999, 109480, 117677, 119043, 119115, 120127, 120562, 121260, 122181, 122869, 123385, 126914, 127671, 128602, 128835, 129601, 130030, 130745, 131640, 132652, 134101, 134111, 134144, 134145, 134821, 135478, 135671, 135759, 137552, 137853, 137871, 138118, 13

In [6]:
path_train_new = os.path.join('./data/train_by_style_type/img/')

tmp_list = []
for k in range(0,289221):
    tmp_list.append(df_cloth_attr_path.iloc[k][0][4:-16])

print(path_train_new)
print(df_cloth_attr_path.iloc[289221][0][4:-16])

tmp_set = set(tmp_list)


def make_folder(folder_name): # 폴더 생성 함수

    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
        
for l in tmp_set:
    make_folder(path_train_new + l)

KeyboardInterrupt: 

In [ ]:
for l in my_style_dict:
    make_folder('./data/train_by_style_type/img_style/' + l + '/')

In [ ]:

new_data_path_style = './data/train_by_style_type/img_style/'
def make_crop(file_path, my_bbox_df):
    img = Image.open(file_path)
    crop_square = (my_bbox_df['x_1'], my_bbox_df['y_1'], my_bbox_df['x_2'], my_bbox_df['y_2'])
    
    crop_img = img.crop(crop_square)
    
    return crop_img

for i in my_style_dict:
    m = 0
    for j in my_style_dict[i]:
        # print(j)
        image = make_crop(basic_path + df_cloth_attr_path.iloc[j][0], df_cloth_bbox.iloc[j])
        img_resize = image.resize((299,299))
        img_resize.save(new_data_path_style + i + '/{}.jpg'.format(m))
        m += 1
    print(i, ' complete')



americana  complete
angeles  complete
art  complete
athletic  complete
audrey  complete
babe  complete
babydoll  complete
barbie  complete
baseball  complete
basic  complete
basquiat  complete
beach  complete
beatles  complete
bed  complete
bella  complete
bike  complete
biker  complete
blah  complete
blurred  complete
boho  complete
bold  complete
boyfriend  complete
brooklyn  complete
brooklyn nets  complete
california  complete
camera  complete
candy  complete
cardio  complete
cat  complete
chic  complete
cities  complete
city  complete
civil  complete
classic  complete
coast  complete
coffee  complete
cute  complete
dainty  complete
daring  complete
dark  complete
darling  complete
defyant  complete
desert  complete
destroyed  complete
devil  complete
doll  complete
doodle  complete
dream  complete
dreamcatcher  complete
dreamer  complete
dynamite  complete
eagle  complete
edge  complete
eiffel  complete
elegant  complete
enchanted  complete
ethereal  complete
everyday  complete
fa

In [ ]:
# 연습
image = make_crop(basic_path + df_cloth_attr_path.iloc[233410][0], df_cloth_bbox.iloc[233410])
img_resize = image.resize((299,299))
img_resize = np.array(img_resize)
plt.imshow(img_resize)